## Code for replicating gao et al research on VUA Sequence Model

In [59]:
# mount drive
from google.colab import drive
ROOT = '/content/drive'
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# add repo directory to path
import os
import sys
from os.path import join 
repo_dir = '/content/drive/MyDrive/Repos/metaphor-detection'
if repo_dir not in sys.path:
    sys.path.append(repo_dir)
print(sys.path)

['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Repos/metaphor-detection']


In [3]:
# directories
# to download glove and elmo vectors see: notebooks/Download_large_data.ipynb
data_dir = repo_dir + '/resources/metaphor-in-context/data/'
glove_dir = repo_dir + '/resources/glove/'
elmo_dir = repo_dir + '/resources/elmo/'

In [4]:
# cd to working directory here if neccesary
%cd $repo_dir
%ls

/content/drive/MyDrive/Repos/metaphor-detection
core/                   install.sh  predictions/  resources/
gao-g-requirements.txt  notebooks/  README.md


Gao code

In [ ]:
# pip install requirements (takes a while)
!cd drive/MyDrive/Repos/metaphor-detection/; pip install -r gao-g-requirements.txt
!pip install --upgrade google-cloud-storage

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from core.gao_files.sequence.util import get_num_lines, get_pos2idx_idx2pos, index_sequence, get_vocab, embed_indexed_sequence, \
    get_word2idx_idx2word, get_embedding_matrix, write_predictions, get_performance_VUAverb_val, \
    get_performance_VUAverb_test, get_performance_VUA_test, get_VUA_POS_summary
from core.gao_files.sequence.util import TextDatasetWithGloveElmoSuffix as TextDataset
from core.gao_files.sequence.util import evaluate
from core.gao_files.sequence.model import RNNSequenceModel

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

import csv
import h5py
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import ast
import pandas as pd

In [8]:
print("PyTorch version:")
print(torch.__version__)
print("GPU Detected:")
print(torch.cuda.is_available())
using_GPU = torch.cuda.is_available()

PyTorch version:
1.10.0+cu111
GPU Detected:
True


In [9]:
"""
1. Data pre-processing
"""
'''
1.1 VUA
get raw dataset as a list:
  Each element is a triple:
    a sentence: string
    a list of labels: 
    a list of pos: 
'''
pos_set = set()
raw_train_vua = []
with open(data_dir + 'VUAsequence/VUA_seq_formatted_train.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        pos_seq = ast.literal_eval(line[4])
        label_seq = ast.literal_eval(line[3])
        assert (len(pos_seq) == len(label_seq))
        assert (len(line[2].split()) == len(pos_seq))
        raw_train_vua.append([line[2], label_seq, pos_seq])
        pos_set.update(pos_seq)

raw_val_vua = []
with open(data_dir + 'VUAsequence/VUA_seq_formatted_val.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        pos_seq = ast.literal_eval(line[4])
        label_seq = ast.literal_eval(line[3])
        assert (len(pos_seq) == len(label_seq))
        assert (len(line[2].split()) == len(pos_seq))
        raw_val_vua.append([line[2], label_seq, pos_seq])
        pos_set.update(pos_seq)

# embed the pos tags
pos2idx, idx2pos = get_pos2idx_idx2pos(pos_set)

for i in range(len(raw_train_vua)):
    raw_train_vua[i][2] = index_sequence(pos2idx, raw_train_vua[i][2])
for i in range(len(raw_val_vua)):
    raw_val_vua[i][2] = index_sequence(pos2idx, raw_val_vua[i][2])
print('size of training set, validation set: ', len(raw_train_vua), len(raw_val_vua))

size of training set, validation set:  6323 1550


In [11]:
raw_train_vua[:2]

[["Ca n't fail to be entertaining .",
  [0, 0, 0, 0, 0, 0, 0],
  [4, 12, 4, 13, 4, 7, 0]],
 ['How much was he going to tell her ?',
  [0, 0, 0, 0, 0, 0, 0, 0, 0],
  [12, 7, 4, 3, 4, 13, 4, 3, 0]]]

In [12]:
"""
2. Data preparation
"""
'''
2. 1
get vocabulary and glove embeddings in raw dataset 
'''
# vocab is a set of words
vocab = get_vocab(raw_train_vua)
# two dictionaries. <PAD>: 0, <UNK>: 1
word2idx, idx2word = get_word2idx_idx2word(vocab)
# glove_embeddings a nn.Embeddings
glove_embeddings = get_embedding_matrix(glove_dir + 'glove.840B.300d.txt',word2idx, idx2word, normalization=False)
# elmo_embeddings
elmos_train_vua = h5py.File(elmo_dir + 'VUA_train.hdf5', 'r')
elmos_val_vua = h5py.File(elmo_dir + 'VUA_val.hdf5', 'r')
# no suffix embeddings for sequence labeling
suffix_embeddings = None

vocab size:  13843


100%|██████████| 2196017/2196017 [01:03<00:00, 34604.87it/s]


Number of pre-trained word vectors loaded:  13404
Embeddings mean:  0.0005707233212888241
Embeddings stdev:  0.3729434907436371


In [13]:
print(len(vocab))
glove_embeddings.weight.shape
# 300d embeddings for the 13843 words in the vocab

13843


torch.Size([13845, 300])

In [14]:
'''
2. 2
embed the datasets
'''
# raw_train_vua: sentence, label_seq, pos_seq
# embedded_train_vua: embedded_sentence, pos, labels
embedded_train_vua = [[embed_indexed_sequence(example[0], example[2], word2idx,
                                      glove_embeddings, elmos_train_vua, suffix_embeddings),
                       example[2], example[1]]
                      for example in raw_train_vua]
embedded_val_vua = [[embed_indexed_sequence(example[0], example[2], word2idx,
                                    glove_embeddings, elmos_val_vua, suffix_embeddings),
                     example[2], example[1]]
                    for example in raw_val_vua]

In [15]:
# embedded_train_vua is a list of lists -- one list per sentence
# each sentence list contains 
#     an array of embeddings (seq_length x embedding_dim)
#     list of pos tag ids
#     list of labels
print(len(embedded_train_vua))
print(len(embedded_train_vua[0]))
print(embedded_train_vua[0][0].shape)
print(embedded_train_vua[0][1])
print(embedded_train_vua[0][2])

6323
3
(7, 1324)
[4, 12, 4, 13, 4, 7, 0]
[0, 0, 0, 0, 0, 0, 0]


In [16]:
'''
2. 3
set up Dataloader for batching
'''
# Separate the input (embedded_sequence) and labels in the indexed train sets.
# embedded_train_vua: embedded_sentence, pos, labels
train_dataset_vua = TextDataset([example[0] for example in embedded_train_vua],
                                [example[1] for example in embedded_train_vua],
                                [example[2] for example in embedded_train_vua])
val_dataset_vua = TextDataset([example[0] for example in embedded_val_vua],
                              [example[1] for example in embedded_val_vua],
                              [example[2] for example in embedded_val_vua])

# Data-related hyperparameters
batch_size = 64
# Set up a DataLoader for the training, validation, and test dataset
train_dataloader_vua = DataLoader(dataset=train_dataset_vua, batch_size=batch_size, shuffle=True,
                              collate_fn=TextDataset.collate_fn)
val_dataloader_vua = DataLoader(dataset=val_dataset_vua, batch_size=batch_size,
                            collate_fn=TextDataset.collate_fn)

In [17]:
"""
3. Model training
"""
'''
3. 1 
set up model, loss criterion, optimizer
'''
# Instantiate the model
# embedding_dim = glove + elmo + suffix indicator
# dropout1: dropout on input to RNN
# dropout2: dropout in RNN; would be used if num_layers!=1
# dropout3: dropout on hidden state of RNN to linear layer
RNNseq_model = RNNSequenceModel(num_classes=2, embedding_dim=300 + 1024, hidden_size=300, num_layers=1, bidir=True,
                                dropout1=0.5, dropout2=0, dropout3=0.1)
# Move the model to the GPU if available
if using_GPU:
    RNNseq_model = RNNseq_model.cuda()
# Set up criterion for calculating loss
loss_criterion = nn.NLLLoss()
# Set up an optimizer for updating the parameters of the rnn_clf
rnn_optimizer = optim.Adam(RNNseq_model.parameters(), lr=0.005)
# Number of epochs (passes through the dataset) to train the model for.
num_epochs = 10

In [ ]:
'''
3. 2
train model
'''
train_loss = []
val_loss = []
performance_matrix = None
val_f1s = []
train_f1s = []
# A counter for the number of gradient updates
num_iter = 0
comparable = []
for epoch in range(num_epochs):
    print("Starting epoch {}".format(epoch + 1))
    for (__, example_text, example_lengths, labels) in train_dataloader_vua:
        example_text = Variable(example_text)
        example_lengths = Variable(example_lengths)
        labels = Variable(labels)
        if using_GPU:
            example_text = example_text.cuda()
            example_lengths = example_lengths.cuda()
            labels = labels.cuda()
        # predicted shape: (batch_size, seq_len, 2)
        predicted = RNNseq_model(example_text, example_lengths)
        batch_loss = loss_criterion(predicted.view(-1, 2), labels.view(-1))
        rnn_optimizer.zero_grad()
        batch_loss.backward()
        rnn_optimizer.step()
        num_iter += 1
        # Calculate validation and training set loss and accuracy every 200 gradient updates
        if num_iter % 200 == 0:
            avg_eval_loss, performance_matrix = evaluate(idx2pos, val_dataloader_vua, RNNseq_model,
                                                         loss_criterion, using_GPU)
            val_loss.append(avg_eval_loss)
            val_f1s.append(performance_matrix[:, 2])
            print("Iteration {}. Validation Loss {}.".format(num_iter, avg_eval_loss))
#             avg_eval_loss, performance_matrix = evaluate(idx2pos, train_dataloader_vua, RNNseq_model,
#                                                          loss_criterion, using_GPU)
#             train_loss.append(avg_eval_loss)
#             train_f1s.append(performance_matrix[:, 2])
#             print("Iteration {}. Training Loss {}.".format(num_iter, avg_eval_loss))

In [ ]:
"""
for additional training
"""
rnn_optimizer = optim.Adam(RNNseq_model.parameters(), lr=0.0001)
for epoch in range(10):
    print("Starting epoch {}".format(epoch + 1))
    for (__, example_text, example_lengths, labels) in train_dataloader_vua:
        example_text = Variable(example_text)
        example_lengths = Variable(example_lengths)
        labels = Variable(labels)
        if using_GPU:
            example_text = example_text.cuda()
            example_lengths = example_lengths.cuda()
            labels = labels.cuda()
        # predicted shape: (batch_size, seq_len, 2)
        predicted = RNNseq_model(example_text, example_lengths)
        batch_loss = loss_criterion(predicted.view(-1, 2), labels.view(-1))
        rnn_optimizer.zero_grad()
        batch_loss.backward()
        rnn_optimizer.step()
        num_iter += 1
        # Calculate validation and training set loss and accuracy every 200 gradient updates
        if num_iter % 200 == 0:
            avg_eval_loss, performance_matrix = evaluate(idx2pos, val_dataloader_vua, RNNseq_model,
                                                         loss_criterion, using_GPU)
            val_loss.append(avg_eval_loss)
            val_f1s.append(performance_matrix[:, 2])
            print("Iteration {}. Validation Loss {}.".format(num_iter, avg_eval_loss))

#             avg_eval_loss, performance_matrix = evaluate(idx2pos, train_dataloader_vua, RNNseq_model,
#                                                          loss_criterion, using_GPU)
#             train_loss.append(avg_eval_loss)
#             train_f1s.append(performance_matrix[:, 2])
#             print("Iteration {}. Training Loss {}.".format(num_iter, avg_eval_loss))
#             comparable.append(get_performance())

print("Training done!")

In [20]:
"""
test on genres by POS tags
"""
print("**********************************************************")
print("Evalutation on test set: ")

raw_test_vua = []
with open(data_dir + 'VUAsequence/VUA_seq_formatted_test.csv', encoding='latin-1') as f:
    lines = csv.reader(f)
    next(lines)
    for line in lines:
        # txt_id	sen_ix	sentence	label_seq	pos_seq	labeled_sentence	genre
        pos_seq = ast.literal_eval(line[4])
        label_seq = ast.literal_eval(line[3])
        assert(len(pos_seq) == len(label_seq))
        assert(len(line[2].split()) == len(pos_seq))
        raw_test_vua.append([line[2], label_seq, pos_seq])
print('number of examples(sentences) for test_set ', len(raw_test_vua))

for i in range(len(raw_test_vua)):
    raw_test_vua[i][2] = index_sequence(pos2idx, raw_test_vua[i][2])

elmos_test_vua = h5py.File(elmo_dir + 'VUA_test.hdf5', 'r')
# raw_train_vua: sentence, label_seq, pos_seq
# embedded_train_vua: embedded_sentence, pos, labels
embedded_test_vua = [[embed_indexed_sequence(example[0], example[2], word2idx,
                                      glove_embeddings, elmos_test_vua, suffix_embeddings),
                       example[2], example[1]]
                      for example in raw_test_vua]

# Separate the input (embedded_sequence) and labels in the indexed train sets.
# embedded_train_vua: embedded_sentence, pos, labels
test_dataset_vua = TextDataset([example[0] for example in embedded_test_vua],
                              [example[1] for example in embedded_test_vua],
                              [example[2] for example in embedded_test_vua])

# Set up a DataLoader for the test dataset
test_dataloader_vua = DataLoader(dataset=test_dataset_vua, batch_size=batch_size,
                              collate_fn=TextDataset.collate_fn)

print("Tagging model performance on VUA test set by POS tags: regardless of genres")
avg_eval_loss, pos_performance_matrix = evaluate(idx2pos, test_dataloader_vua, RNNseq_model, loss_criterion, using_GPU)

**********************************************************
Evalutation on test set: 
number of examples(sentences) for test_set  2694
Tagging model performance on VUA test set by POS tags: regardless of genres


/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:239: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_text = Variable(eval_text, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:240: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_lengths = Variable(eval_lengths, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:241: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_labels = Variable(eval_labels, volatile=True)


------------------------------
total_eval_loss.shape torch.Size([])
PRFA performance for  PUNCT nan 0.0 nan 99.94099134539732
PRFA performance for  INTJ nan 0.0 nan 99.49748743718592
PRFA performance for  SYM nan nan nan 100.0
PRFA performance for  PRON nan 0.0 nan 99.84829329962074
PRFA performance for  VERB 69.72176759410802 67.72655007949126 68.70967741935485 88.20907617504052
PRFA performance for  DET 89.49771689497717 93.55608591885442 91.48191365227538 98.22729480330257
PRFA performance for  PROPN 66.66666666666667 12.903225806451612 21.62162162162162 98.47769028871392
PRFA performance for  ADJ 67.12328767123287 54.34380776340111 60.06128702757916 90.13871374527112
PRFA performance for  CCONJ nan nan nan 100.0
PRFA performance for  NUM nan nan nan 100.0
PRFA performance for  ADP 88.3054892601432 88.62275449101796 88.4638374178123 92.71698113207547
PRFA performance for  X nan nan nan 100.0
PRFA performance for  ADV 72.58064516129032 55.32786885245902 62.79069767441861 95.284409077

/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:372: RuntimeWarning: invalid value encountered in double_scalars
  precision = 100 * grid[1, 1] / np.sum(grid[1])
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:373: RuntimeWarning: invalid value encountered in double_scalars
  recall = 100 * grid[1, 1] / np.sum(grid[:, 1])


In [21]:
# compute POS summary stats over the VUA train data
pos_cnts, pos_m_rate = get_VUA_POS_summary(raw_train_vua, idx2pos)
POS_selects = ["VERB", "NOUN", "ADP", "ADJ", "PART"]
pos_cnts_list = [pos_cnts[POS] for POS in POS_selects]
pos_meta_rate_list = [pos_m_rate[POS] for POS in POS_selects]

In [22]:
print("Breakdown of performance on the VUA sequence labeling test set by POS tags: ")
pos_table = pd.DataFrame(pos_performance_matrix[[pos2idx['VERB'],pos2idx['NOUN'],
                                                 pos2idx['ADP'],pos2idx['ADJ'],pos2idx['PART']],:],
                         columns = ["Pr", "Re","F1","Acc"],
                         index=["VERB","NOUN","ADP","ADJ","PART"])
pos_table['Count'] = pos_cnts_list
pos_table['% metaphor'] = pos_meta_rate_list
pos_table

Breakdown of performance on the VUA sequence labeling test set by POS tags: 


,Pr,Re,F1,Acc,Count,% metaphor
VERB,69.721768,67.726550,68.709677,88.209076,20917,0.180953
NOUN,69.498069,55.770720,61.882252,89.671635,20514,0.136151
ADP,88.305489,88.622754,88.463837,92.716981,13310,0.280391
ADJ,67.123288,54.343808,60.061287,90.138714,9673,0.115269
PART,57.553957,53.691275,55.555556,91.250854,2966,0.100809


In [61]:
seq_test_pred = write_predictions(raw_test_vua, test_dataloader_vua,
                                  RNNseq_model, using_GPU,
                                  data_dir + 'VUAsequence/VUA_seq_formatted_test.csv',
                                  write_out_preds=True,
                                  pred_filename="vua_seq_bilstm_pred.txt")

/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:330: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_text = Variable(eval_text, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:331: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_lengths = Variable(eval_lengths, volatile=True)
/content/drive/MyDrive/Repos/metaphor-detection/core/gao_files/sequence/util.py:332: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  eval_labels = Variable(eval_labels, volatile=True)


In [ ]:
!git add notebooks/seq

In [ ]:
macro_avg_performance, overall_verb_performance = get_performance_VUAverb_test(data_dir, seq_test_pred)

Tagging model performance on test-verb: genre
news Precision, Recall, F1, Accuracy:  74.18738049713193 69.40966010733453 71.71903881700555 75.0814332247557
fiction Precision, Recall, F1, Accuracy:  57.67918088737201 61.904761904761905 59.717314487632514 83.53790613718411
academic Precision, Recall, F1, Accuracy:  75.81168831168831 73.1974921630094 74.48165869218501 74.58300238284353
conversation Precision, Recall, F1, Accuracy:  60.72874493927125 51.54639175257732 55.76208178438662 88.10594702648676
Tagging model performance on test-verb: regardless of genre
Precision, Recall, F1, Accuracy:  69.9225729600953 66.66666666666667 68.25581395348837 81.40643623361144


In [ ]:
macro_F1 = macro_avg_performance[2]
classification_performance = np.append(overall_verb_performance, macro_F1)

In [ ]:
get_performance_VUA_test(data_dir,seq_test_pred)

Tagging model performance on test-sequence: genre
news Precision, Recall, F1, Accuracy:  78.00751879699249 66.50641025641026 71.79930795847751 92.06426484907497
fiction Precision, Recall, F1, Accuracy:  69.82310093652445 66.17357001972387 67.94936708860759 94.22761262082801
academic Precision, Recall, F1, Accuracy:  82.87829708468301 76.14795918367346 79.37070684688678 93.16195372750643
conversation Precision, Recall, F1, Accuracy:  67.41803278688525 65.4726368159204 66.43109540636043 94.98869630746044
Tagging model performance on test-sequence: regardless of genre
Precision, Recall, F1, Accuracy:  76.65964172813487 69.91830850552618 73.13395325458657 93.60837070254111


array([74.5317374 , 68.57514407, 71.38761933, 93.61063188])

In [ ]:
gao_scores_verb = [68.0, 68.3, 68.2, 80.9, 65.4]
our_scores_verb = classification_performance
our_scores_verb = [round(score,1) for score in our_scores_verb]
all_scores_verb = [gao_scores_verb, our_scores_verb]
all_scores_verb_df = pd.DataFrame(all_scores_verb, columns= ['P', 'R', 'F1', 'Acc', 'MaF1'], index=['Gao et al', 'US'])
print("VUA seq model: classification task\n")
all_scores_verb_df

VUA seq model: classification task



,P,R,F1,Acc,MaF1
Gao et al,68.0,68.3,68.2,80.9,65.4
US,69.9,66.7,68.3,81.4,65.4
